# *Machine Learning* em Reanimação Neonatal
Projeto 1 da Disciplina de Aprendizado de Máquina - PPgEEC/UFRN

## Etapa 3 - DATA SEGREGATION

Neste notebook será realizada a divisão do *dataset* em **conjunto de treino** e **conjunto de teste** para, posteriormente, ser utilizado no algoritmo de **Decision Tree**.

## 1.  Configurações Iniciais

In [1]:
import logging
import os
import tempfile
import pandas as pd
import wandb
from sklearn.model_selection import train_test_split

In [2]:
!wandb login c4d070b0b80b59891761b50eb9912749a4af585e --relogin

wandb: Appending key for api.wandb.ai to your netrc file: /Users/ericcalasans/.netrc


## 2.  Segregação dos Dados

Criação de variáveis globais de apoio

In [3]:
# Tamanho do conjunto de teste(fração do dataset que é utilizada como dados de teste)
test_size = 0.3

# Coeficiente de aleatoriedade
seed = 1618

# Variável alvo(target)
stratify = 'reanimacao'

# Artefato de entrada
input_artifact = 'mlreanimacao/clean_data.csv:latest'

# Tipo do artefato
artifact_type = 'segregated_data'

Configurações do objeto de log - uso do módulo **logging**

In [4]:
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# Criação de um objeto para produzir os logs
logger = logging.getLogger()

Captura do artefato

In [5]:
run = wandb.init(project='mlreanimacao', job_type='split_data')

# Registra um log desta ação
logger.info("Baixando artefato...")
artifact = run.use_artifact(input_artifact)
artifact_file = artifact.file()
df_to_split = pd.read_csv(artifact_file)

23-05-2022 22:04:34 Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ecalasans. Use `wandb login --relogin` to force relogin


23-05-2022 22:04:44 Baixando artefato...


Divisão do *dataset*

In [6]:
conjunto = {}

# Conjunto de treino(train) e de teste(test)
conjunto["trian"], conjunto["test"] = train_test_split(
    df_to_split,
    test_size=test_size,
    random_state=seed,
    stratify=df_to_split['reanimacao']
)

## 3. Criação dos Artefatos

Serão criados artefatos separados para os conjuntos de treino e teste.

In [8]:
with tempfile.TemporaryDirectory() as temp_dir:
    for conj, df in conjunto.items():
        artifact_name = f"{conj}.csv"
        temp_path = os.path.join(temp_dir, artifact_name)
        logger.info(f"Salvando o {conj} dataset para o {artifact_name}")
        df.to_csv(temp_path, index=False)
        artifact = wandb.Artifact(
            name=artifact_name,
            type=artifact_type,
            description=f"{conj} parte do dataset {input_artifact} "
        )
        artifact.add_file(temp_path)
        logger.info("Log do artefato")
        run.log_artifact(artifact)
        artifact.wait()  # Espera o artefato ser logado
    

23-05-2022 22:05:20 Salvando o trian dataset para o trian.csv
23-05-2022 22:05:20 Log do artefato
23-05-2022 22:05:26 Salvando o test dataset para o test.csv
23-05-2022 22:05:26 Log do artefato


In [9]:
# Termina a execução do wandb
run.finish()